# Laboratorio de Robótica Industrial

### David Rozo Osorio, I.M, M.Sc.

## Introducción a Sistemas de Control Embebidos con *Robot Operating System 1* (ROS1)

Objetivo: operación de hardware y control de bajo nivel con intermediación de *Robot Operating System* (ROS).

Procedimiento:

1. Descripción del Contexto Operativo.
2. Hello World! - Arduino y ROS
3. Descripción del Paquete RobotCar_G00

    

## Descripción del Contexto Operativo.

Un robot móvil en configuración diferencial consiste en 2 ruedas actuadas comunmente ubicadas en direcciones opuestas en el cuerpo del robot. El movimiento y la dirección del dispositivo se obtienen a partir del movimiento realtivo de las ruedas y la relación entre estas. Una ventaja del este dispotivo consiste en que no requiere de una entrada de dirección adicional. Para balancear el robot es comun adicionar una rueda o apoyo sin fricción en un extremo del cuerpo. A continuación se presenta un esquema del robot descirto.

<img src="Imag/03_Robot.jpg" width=300 height=300 />

**Descripción de movimiento**:

- Movimiento lineal puro (positivo o negativo): se obtiene al comandar los motores en la misma dirección a la misma velocidad.
- Movimiento angular puro: se obtiene al comandar los motores en direcciones opuestas pero a la misma velocidad.
- Movimiento combinado (traslación y rotación): se obtiene al comandar los motores a diferentes velocidades. 

**Modelo cinemático**:

- **Cinemática Directa Diferencial**: describe la velocidad lineal y angular del vehículo a partir del la velocidad angular de las ruedas. 

$$\left[\begin{matrix} \dot{x}\\ \dot{y} \\ \dot{\theta} \end{matrix}\right] = \left[\begin{matrix} \dfrac{r}{2} & \dfrac{r}{2} \\ 0 & 0 \\ \dfrac{r}{2b} & \dfrac{r}{2b} \end{matrix}\right] \left[\begin{matrix} \dot{\phi}_{r}\\ \dot{\phi}_{l} \end{matrix}\right]$$

- **Cinemática Inversa Diferencial**: describe la velocidad angular de las ruedas a partir de la velocidad lineal y angular del vehículo.

$$ \left[\begin{matrix} \dot{\phi}_{r}\\ \dot{\phi}_{l} \end{matrix}\right] = \left[\begin{matrix} \dfrac{1}{r} & 0 & \dfrac{b}{r} \\ \dfrac{1}{r} & 0 & \dfrac{-b}{r} \end{matrix}\right] \left[\begin{matrix} \dot{x}\\ \dot{y} \\ \dot{\theta} \end{matrix}\right]$$

**Esquemático Cinemático**

<img src="Imag/03_DiagramKinematics.jpg" width=200 height=200 />

**Equemático de Control**

<img src="Imag/02_Diagrama.jpg" width=600 height=600 />

## Configuración de ROS

**Nota**: la presente guía asume que actualemente se tiene la distribución ROS-Noetic instalada y funcional.

El paquete `rosserial_arduino` provee una protocolo de comunicación que opera sobre las estructur UART del Arduino. Este puente convierte el Arduino en un Nodo ROS que puede publicar y subscribirse a mensajes ROS directamente.

### Instalación desde Fuente a una estación ROS

```bash
$ cd catkin_ws/src
catkin_ws/src $ git clone https://github.com/ros-drivers/rosserial.git
catkin_ws/src $ cd .. 
catkin_ws $ catkin_make
catkin_ws $  catkin_make install
```
El procedimiento anterior clona de github el repositorio `rosserial`, y genera los `rosserial_msgs` necesarios para la comunicación, adicionalmente, crea las liberias necesarias.

## Hello World ROS!

- Encabezado conjunto de librerías

```cpp
#include <ros.h>
#include <std_msgs/String.h>
```

- Definición de una instacia u objeto *ROS-Node-Handler*

```cpp
ros::NodeHandle nh;
```

- Definición de la instacia del **Publicador**. El objeto se llama  `chatter` y el *topic* también toma este mismo nombre. El 2do parámetros para el **Publicador** es una referencia a una instacia del tipo de mensaje a enviar. 

```cpp
std_msgs::String str_msg;
ros::Publisher chatter("chatter", &str_msg);

// definción del mensaje estatico
char hello[] = "hello world ROS!";
```

- Definción de una instacia del **Subscriptor**. El objeto se llama `sub` y el *topic* se llama `toggle_led`. El 2do parámetro para el **Subscriptor** es el nombre de la función encargada de administrar los mensajes entrantes.

```cpp
void messageCb( const std_msgs::Int32& toggle_msg){
  digitalWrite(13, toggle_msg.data);   // blink the led
}

ros::Subscriber<std_msgs::Int32> sub("toggle_led", messageCb );
```




- En la función `setup`, se debe inicializar el *ROS-Handler* y se deben publicitar todos los *topics* publicadores y se deben subscribir todos los *topics* subscriptores. 

```cpp
void setup()
{
  nh.initNode();
  nh.advertise(chatter);
}
```

- En la función `loop`, se publica la información y se llama al método `spinOnce()` del *ROS-Handler*, esta función se encarga de administrar todas las comunicaciones. 

```cpp
void loop()
{
   // msg asigned
  str_msg.data = hello;
  //  msg pub
  chatter.publish( &str_msg );
  // callback all
  nh.spinOnce(); 
  // sample time
  delay(1000);
}
```